In [ ]:
!pip install stanza

In [3]:
import stanza

# Download Swedish model (only first time)
stanza.download("sv")

# Initialize the Swedish pipeline
nlp = stanza.Pipeline(lang="sv", processors="tokenize,pos,lemma,depparse")

# Example text
text = "Exempelvis är tjugoett och tio relativt prima."

# Run the pipeline
doc = nlp(text)

# Print dependency relations
for sentence in doc.sentences:
    for word in sentence.words:
        print(f"{word.text}\t{word.lemma}\t{word.upos}\thead={word.head}\tdeprel={word.deprel}")


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: sv (Swedish) ...


INFO:stanza:Downloaded file to /root/stanza_resources/sv/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: sv (Swedish):
| Processor | Package            |
----------------------------------
| tokenize  | talbanken          |
| pos       | talbanken_charlm   |
| lemma     | talbanken_nocharlm |
| depparse  | talbanken_charlm   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


Exempelvis	exempelvis	ADV	head=7	deprel=advmod
är	vara	AUX	head=7	deprel=cop
tjugoett	tjugoett	ADJ	head=7	deprel=nsubj
och	och	CCONJ	head=5	deprel=cc
tio	tio	NUM	head=3	deprel=nummod
relativt	relativt	ADV	head=7	deprel=advmod
prima	prim	ADJ	head=0	deprel=root
.	.	PUNCT	head=7	deprel=punct


In [4]:
from PIL import Image, ImageDraw, ImageFont
from textwrap import wrap
from pathlib import Path

# --- helpers ---------------------------------------------------------------

def _find_font():
    """
    Try to find a decent sans-serif font cross-platform.
    Fallback to PIL's default if none are found.
    """
    candidates = [
        "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",               # Linux
        "/System/Library/Fonts/Supplemental/Arial Unicode.ttf",          # macOS
        "/System/Library/Fonts/Helvetica.ttc",                           # macOS
        "C:/Windows/Fonts/arial.ttf",                                    # Windows
    ]
    for p in candidates:
        if Path(p).exists():
            try:
                return ImageFont.truetype(p, 80)
            except Exception:
                pass
    return ImageFont.load_default()

def _fit_text(draw, text, max_width, max_height, base_font, line_spacing=1.15):
    """
    Binary-search a font size that fits the block (with wrapping).
    Returns (font, lines).
    """
    low, high = 12, 220
    best = (base_font, [text])
    while low <= high:
        mid = (low + high) // 2
        font = ImageFont.truetype(base_font.path, mid) if hasattr(base_font, "path") else base_font
        # greedy wrap by character width
        words = text.split()
        lines = []
        line = ""
        for w in words:
            test = (line + " " + w).strip()
            w_px, _ = draw.textbbox((0, 0), test, font=font)[2:]
            if w_px <= max_width:
                line = test
            else:
                if line:
                    lines.append(line)
                line = w
        if line:
            lines.append(line)
        # measure height
        ascent, descent = font.getmetrics()
        line_h = ascent + descent
        total_h = int(len(lines) * line_h * line_spacing)
        max_line_w = max(draw.textbbox((0, 0), ln, font=font)[2] for ln in lines) if lines else 0
        if total_h <= max_height and max_line_w <= max_width:
            best = (font, lines)
            low = mid + 1
        else:
            high = mid - 1
    return best

def _palette(style):
    """
    Simple palettes: 'dark', 'light', 'sunset'
    """
    if style == "light":
        return {"bg": (248, 249, 251), "fg": (20, 24, 31), "shadow": (255, 255, 255, 80)}
    if style == "sunset":
        return {"bg": (252, 236, 210), "fg": (28, 25, 23), "shadow": (255, 255, 255, 60)}
    # default: dark
    return {"bg": (18, 21, 26), "fg": (240, 242, 245), "shadow": (0, 0, 0, 90)}

# --- main ------------------------------------------------------------------

def render_phrase_images(
    phrases,
    out_dir="phrase_images",
    size=(1600, 900),
    style="dark",
    margin_ratio=0.08,
    line_spacing=1.15,
    add_shadow=True,
    round_corners=True,
    filename_prefix="img_",
):
    """
    Render each phrase to a PNG with clean typography.

    phrases: list[str]
    out_dir: output folder
    size: (width, height) in pixels
    style: 'dark' | 'light' | 'sunset'
    margin_ratio: fraction of min(width,height) for padding
    line_spacing: multiplier for line height
    add_shadow: draw a subtle text shadow
    round_corners: rounded border mask for gentle poster feel
    filename_prefix: prefix for saved files
    """
    out = Path(out_dir)
    out.mkdir(parents=True, exist_ok=True)

    W, H = size
    palette = _palette(style)
    base_font = _find_font()

    saved = []
    for i, phrase in enumerate(phrases, 1):
        img = Image.new("RGBA", (W, H), palette["bg"])
        draw = ImageDraw.Draw(img)

        pad = int(min(W, H) * margin_ratio)
        max_w = W - pad * 2
        max_h = H - pad * 2

        font, lines = _fit_text(draw, phrase, max_w, max_h, base_font, line_spacing=line_spacing)

        # Measure block to center it
        ascent, descent = font.getmetrics()
        line_h = ascent + descent
        block_h = int(len(lines) * line_h * line_spacing)
        y = (H - block_h) // 2

        # Shadow layer (optional)
        if add_shadow:
            shadow = Image.new("RGBA", (W, H), (0, 0, 0, 0))
            sdraw = ImageDraw.Draw(shadow)
            for ln in lines:
                w_ln = draw.textbbox((0, 0), ln, font=font)[2]
                x = (W - w_ln) // 2
                sdraw.text((x+3, y+3), ln, font=font, fill=palette["shadow"])
                y += int(line_h * line_spacing)
            img = Image.alpha_composite(img, shadow)
            y = (H - block_h) // 2  # reset

        # Foreground text
        for ln in lines:
            w_ln = draw.textbbox((0, 0), ln, font=font)[2]
            x = (W - w_ln) // 2
            draw.text((x, y), ln, font=font, fill=palette["fg"])
            y += int(line_h * line_spacing)

        # Rounded mask (subtle)
        if round_corners:
            from PIL import ImageFilter, ImageChops
            radius = int(min(W, H) * 0.03)
            mask = Image.new("L", (W, H), 0)
            mdraw = ImageDraw.Draw(mask)
            mdraw.rounded_rectangle([2, 2, W-2, H-2], radius=radius, fill=255)
            img.putalpha(255)
            img = Image.composite(img, Image.new("RGBA", (W, H), palette["bg"]), mask)
            # Add a soft outer border
            border = mask.filter(ImageFilter.GaussianBlur(2))
            img = Image.alpha_composite(Image.new("RGBA", (W, H), palette["bg"]), img)
            img.putalpha(255)

        # Save
        fname = f"{filename_prefix}{i:02d}.png"
        path = out / fname
        img.convert("RGB").save(path, "PNG")
        saved.append(str(path))

    return saved

# --- example usage ---------------------------------------------------------

if __name__ == "__main__":
    phrases = [
        "Exempelvis är tjugoett och tio relativt prima.",
        "Where’s och?",
        "Recovering Speech from the Record"
    ]
    files = render_phrase_images(phrases, style="light", size=(1920, 1080))
    print("Saved:", files)
